<a href="https://colab.research.google.com/github/futartup/S8-assignment/blob/master/transform_train_test_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import glob
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

# **Create transformation pipeline for training**

In [0]:
import albumentations as A
from albumentations.pytorch import ToTensor


class AlbumentationDataSet():
  def __init__(self, train=True):
    if train:
      self.albu_aug = A.Compose(
                      [
                          A.HorizontalFlip(p = 0.5), # apply horizontal flip to 50% of images
                          A.OneOf(
                              [
                                  # apply one of transforms to 50% of images
                                  A.RandomContrast(always_apply=False, p=1.0, limit=(-0.25999999046325684, 0.17000000178813934)), # apply random contrast
                                  A.RandomGamma(always_apply=False, p=0.25, gamma_limit=(80, 120)), # apply random gamma
                                  A.RandomBrightness(always_apply=False, p=1.0, limit=(-0.3700000047683716, 0.35999998450279236)), # apply random brightness
                                  A.RGBShift(always_apply=True, p=0.25, r_shift_limit=(-20, 20), g_shift_limit=(-20, 20), b_shift_limit=(-20, 20)),
                                  #A.Blur(always_apply=True, p=0.25, blur_limit=(3, 7)),
                            ],
                              p = 0.5
                          ),
                          # A.OneOf(
                          #     [
                          #         # apply one of transforms to 50% images
                          #         A.ElasticTransform(
                          #             alpha = 120,
                          #             sigma = 120 * 0.05,
                          #             alpha_affine = 120 * 0.03
                          #         ),
                          #         A.GridDistortion(),
                          #         A.OpticalDistortion(
                          #             distort_limit = 2,
                          #             shift_limit = 0.5
                          #         ),
                          #     ],
                          #     p = 0.5
                          # ),
                          A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, always_apply=True, p=1.0),
                          A.Normalize(
                            mean=[0.4914, 0.4822, 0.4465],
                            std=[0.2023, 0.1994, 0.2010],
                            always_apply=True,
                            ),
                          
                          ToTensor() # convert the image to PyTorch tensor
                      ],
                      p = 1
                  )
      
  def __len__(self):
    return len(self.image_list)

  def __call__(self, image):
    # image = plt.imread(numpy.asarray(self.image_list[i][0]))
    # image = Image.fromarray(image).convert('RGB')
    # image = self.albu_aug(image=np.array(image))['image']
    # #image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    # return image
    image = np.array(image)
    image = self.albu_aug(image=image)['image']
    return image
  

In [0]:
# transform_train = transforms.Compose([augmentation_pipeline,
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
#transform_train = augmentation_pipeline
# Get the image list


# **Apply Transformations on the test set**

In [0]:
# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# **Get the loader for the train data**

In [0]:
def get_train_loader():
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=AlbumentationDataSet())
  #alb_dataset = AlbumentationDataSet(image_list=trainset)
  #train_loader = DataLoader(dataset=alb_dataset, batch_size=64, shuffle=True, num_workers=2)
  train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
  return train_loader


# **Get the loader for the test data**

In [0]:
def get_test_loader():  
  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
  test_loader = DataLoader(testset, batch_size=64,
                           shuffle=False, num_workers=2)
  return test_loader

In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')